In [25]:
import sys
import pandas as pd
import yfinance as yf
from datetime import datetime, time, timedelta, date
import holidays

In [26]:
def is_us_holiday(date_obj):
    us_holidays = holidays.US()
    return date_obj in us_holidays

In [27]:
def get_next_working_day(date_obj):
    while True:
        date_obj += timedelta(days=1)
        if date_obj.weekday() < 5 and not is_us_holiday(date_obj):
            return date_obj

In [28]:
def get_open_close_prices(symbol, timestamp):
    try:
        print("symbol: ", symbol)
        print("timestamp: ", timestamp)

        date_obj = datetime.strptime(timestamp, "%a %b %d %H:%M:%S +0000 %Y")
        print(date_obj)
        start_date = datetime(date_obj.year, date_obj.month, date_obj.day, 0, 0, 0)
        end_date = datetime(date_obj.year, date_obj.month, date_obj.day, 16, 0, 0)

        market_closing_time = time(16, 0, 0)

        if date_obj.time() <= market_closing_time:
            if is_us_holiday(date_obj.date()):
                print("Tweet made on a public holiday")
                date_obj = get_next_working_day(date_obj.date())
                print("Updated date object for the next working date: ", date_obj)

            # Check if the date falls on a weekend (Saturday or Sunday) or after Friday market closing
            if date_obj.weekday() == 5:  # Saturday
                print("Tweet made on Saturday")
                date_obj += timedelta(days=2)
                print("Updating date_obj to Monday: ", date_obj)
            elif date_obj.weekday() == 6:  # Sunday
                print("Tweet made on Saturday")
                date_obj += timedelta(days=1)
                print("Updating date_obj to Monday: ", date_obj)

            start_date = datetime(date_obj.year, date_obj.month, date_obj.day, 0, 0, 0)
            end_date = datetime(date_obj.year, date_obj.month, date_obj.day, 16, 0, 0)
            print("start_date same day: ", start_date)
            print("end_date same day: ", end_date)

        elif date_obj.time() > market_closing_time:
            if is_us_holiday(date_obj.date()):
                print("Tweet on Public Holiday, Updating to fetch price on Working day")
                date_obj = get_next_working_day(date_obj.date())
                start_date = datetime(date_obj.year, date_obj.month, date_obj.day, 0, 0, 0)
                end_date = datetime(date_obj.year, date_obj.month, date_obj.day, 16, 0, 0)
                print("start_date next day: ", start_date)
                print("end_date next day: ", end_date)
            else:
                # Check if the date falls on a weekend (Saturday or Sunday) or after Friday market closing
                if date_obj.weekday() == 5:
                    print("Tweet made on Saturday")
                    date_obj += timedelta(days=2)
                    print("Updating date_obj to Monday: ", date_obj)
                elif date_obj.weekday() == 6:
                    print("Tweet made on Sunday")
                    date_obj += timedelta(days=1)
                    print("Updating date_obj to Monday: ", date_obj)
                elif date_obj.weekday() == 4 and date_obj.time() > market_closing_time:  # After Friday market closing
                    print("Tweet made on Friday, After market closing")
                    date_obj += timedelta(days=3)
                    print("Updating date_obj to Monday: ", date_obj)
                else:
                    print("Tweet made after market closing on ", date_obj)
                    date_obj += timedelta(days=1)
                    print("Updating date_obj to next day: ", date_obj)

                start_date = datetime(date_obj.year, date_obj.month, date_obj.day, 0, 0, 0)
                end_date = datetime(date_obj.year, date_obj.month, date_obj.day, 16, 0, 0)
                print("start_date next day: ", start_date)
                print("end_date next day: ", end_date)

        data = yf.download(symbol, start=start_date, end=end_date)
        print(data)
        if data.empty:
            open_price, close_price = None, None
        else:
            open_price = data['Open'].iloc[0]
            close_price = data['Close'].iloc[0]

    except Exception as e:
        open_price, close_price = None, None

    return open_price, close_price



In [29]:
chunk_size = 20
output_file = '/StockMarketCommunityDetector/result/output_data.csv'

In [30]:
with pd.read_csv('/StockMarketCommunityDetector/result/stockerbot-export.csv', usecols=['timestamp', 'source', 'symbols', 'company_names'], chunksize=chunk_size) as reader:
    for chunk in reader:
        open_prices = []
        close_prices = []

        for index, row in chunk.iterrows():
            symbol = row['symbols'].strip()
            timestamp = row['timestamp']

            open_price, close_price = get_open_close_prices(symbol, timestamp)

            open_prices.append(open_price)
            close_prices.append(close_price)


        chunk['open'] = open_prices
        chunk['close'] = close_prices


        chunk.to_csv(output_file, mode='a', header=False, index=False)


symbol:  GS
timestamp:  Wed Jul 18 21:33:26 +0000 2018
2018-07-18 21:33:26
Tweet made after market closing on  2018-07-18 21:33:26
Updating date_obj to next day:  2018-07-19 21:33:26
start_date next day:  2018-07-19 00:00:00
end_date next day:  2018-07-19 16:00:00
[*********************100%%**********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2018-07-19  231.110001  231.990005  228.979996  229.630005  204.067139   

             Volume  
Date                 
2018-07-19  4638500  
symbol:  M
timestamp:  Wed Jul 18 22:22:47 +0000 2018
2018-07-18 22:22:47
Tweet made after market closing on  2018-07-18 22:22:47
Updating date_obj to next day:  2018-07-19 22:22:47
start_date next day:  2018-07-19 00:00:00
end_date next day:  2018-07-19 16:00:00
[*********************100%%**********************]  1 of 1 completed
                 Open   High    Low      Clo


1 Failed download:
['BTC']: Exception("%ticker%: Data doesn't exist for startDate = 1531972800, endDate = 1532030400")



Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
symbol:  ORCL
timestamp:  Wed Jul 18 23:00:01 +0000 2018
2018-07-18 23:00:01
Tweet made after market closing on  2018-07-18 23:00:01
Updating date_obj to next day:  2018-07-19 23:00:01
start_date next day:  2018-07-19 00:00:00
end_date next day:  2018-07-19 16:00:00
[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2018-07-19  48.400002  48.720001  48.209999  48.439999  44.450974  16656500
symbol:  FB-GOOGL-GOOG
timestamp:  Wed Jul 18 23:04:00 +0000 2018
2018-07-18 23:04:00
Tweet made after market closing on  2018-07-18 23:04:00
Updating date_obj to next day:  2018-07-19 23:04:00
start_date next day:  2018-07-19 00:00:00
end_date next day:  2018-07-19 16:00:00
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['FB-GOOGL-GOOG']: Exception('%ticker%: No timezone found, symbol may be delisted')



Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
symbol:  HRS
timestamp:  Wed Jul 18 23:06:58 +0000 2018
2018-07-18 23:06:58
Tweet made after market closing on  2018-07-18 23:06:58
Updating date_obj to next day:  2018-07-19 23:06:58
start_date next day:  2018-07-19 00:00:00
end_date next day:  2018-07-19 16:00:00
[*********************100%%**********************]  1 of 1 completed

...


1 Failed download:
['JEC']: Exception('%ticker%: No timezone found, symbol may be delisted')



Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
symbol:  OKE
timestamp:  Tue Jul 10 20:53:03 +0000 2018
2018-07-10 20:53:03
Tweet made after market closing on  2018-07-10 20:53:03
Updating date_obj to next day:  2018-07-11 20:53:03
start_date next day:  2018-07-11 00:00:00
end_date next day:  2018-07-11 16:00:00
[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2018-07-11  70.860001  71.139999  69.730003  70.309998  48.185741  1960800
symbol:  CY
timestamp:  Tue Jul 10 20:55:02 +0000 2018
2018-07-10 20:55:02
Tweet made after market closing on  2018-07-10 20:55:02
Updating date_obj to next day:  2018-07-11 20:55:02
start_date next day:  2018-07-11 00:00:00
end_date next day:  2018-07-11 16:00:00
[*********************100%%**********************]  1 of 1 completed


1 Failed download:
['CY']: Exception('%ticker%: No timezone found, symbol may be delisted')



Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
symbol:  OKE
timestamp:  Tue Jul 10 20:56:18 +0000 2018
2018-07-10 20:56:18
Tweet made after market closing on  2018-07-10 20:56:18
Updating date_obj to next day:  2018-07-11 20:56:18
start_date next day:  2018-07-11 00:00:00
end_date next day:  2018-07-11 16:00:00
[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2018-07-11  70.860001  71.139999  69.730003  70.309998  48.185738  1960800
symbol:  AIZ
timestamp:  Tue Jul 10 20:57:21 +0000 2018
2018-07-10 20:57:21
Tweet made after market closing on  2018-07-10 20:57:21
Updating date_obj to next day:  2018-07-11 20:57:21
start_date next day:  2018-07-11 00:00:00
end_date next day:  2018-07-11 16:00:00
[*********************100%%**********************]  1 of 1 completed
              Open       

KeyboardInterrupt: 